Install python packages which are reuqired in thei example. Here we are using "requirements.txt" file which contains a list of packages. 

In [ ]:
pip install -r requirements.txt --ignore-installed

Refrencing the libraries from the pythin packages

In [ ]:
from typing import List
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from io import BytesIO
from huggingface_hub import hf_hub_download
from langchain.vectorstores import FAISS
import chainlit as cl
import io
import os
import PyPDF2
import urllib.request


Set global variables for folders where documents will be recided and Vector store will be created

In [5]:
DATA_PATH = 'localdata'
DB_FAISS_PATH = 'vectorstore'

Creating required folders. Here two folders are doing to be created. There will be a folder called "localdata" where documents can be stored. Anoother folder "vectorstore" will also be created, this folder will store vectors. At this end, we are also going to download a PDF file about DevSecOps from internet to use as an example.

In [ ]:
# Parent Directory path 
parent_dir = "./"

# Create folder for localdata
directory = DATA_PATH
path = os.path.join(parent_dir, directory) 
if not os.path.exists(directory):
    os.mkdir(path) 
    
# PCreate folder for VectorStore
directory = DB_FAISS_PATH
path = os.path.join(parent_dir, directory) 
if not os.path.exists(directory):
    os.mkdir(path) 
    
urllib.request.urlretrieve("https://insights.sei.cmu.edu/documents/1287/2020_005_001_638589.pdf", "localdata/devsecops.pdf")

Setting up a global variable to store the prompt template

In [11]:
custom_prompt_template = """Use the following context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

Function which is responsible to download embedding model from huggingface and store the downloaded model to a folder called "models". Whenever we will need the embedding model, we will call this method

In [12]:
def hugging_embedding():
    
    # you can use a different embedding by replacing with model_name = sentence-transformers/all-MiniLM-L6-v2. system will download it during first execution
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5",
                                       model_kwargs={'device': 'cpu'},
                                       cache_folder="./models")
    return embeddings


Function which is responsible to create the vector store. Multiple operation are being taken place in this function. 
a)Load the pdf files from localdata folder into a pdd loader
b)Divide pdf file into pages
c)Split the text present on each page into multiple chunks based on the chunk size of 400 with overlap of 80 characters.
d)Use embedding model to calcualltor vectors for the words and store vectors in FAISS Vector Database locally.

In [13]:
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,
                                                   chunk_overlap=80)
    texts = text_splitter.split_documents(documents)
    
    # you can use a different embedding by replacing with model_name = sentence-transformers/all-MiniLM-L6-v2. system will download it during first execution
    embeddings = hugging_embedding()

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

This function is responsbile to download Large Lanaguage Model (LLM) from hugging face and store it in a folder called "models"

In [14]:
def downloadmodel(model_id,model_filename,model_path_cache):
    model_path = hf_hub_download(             
                                 repo_id=model_id,             
                                 filename=model_filename,             
                                 resume_download=True,             
                                 cache_dir=model_path_cache,         
                                 )
    return model_path


This function will load the LLM into memory so that later part of the application can use it. This is one of the most imporatant method because through 
this we can set some of the hyper paramaters for the LLM. With these paramaters we can config and chane the behaviour of LLM

In [15]:
def load_llm():

    #uncomment to allow the UI to download a different model during first execution
    mdpath = downloadmodel("TheBloke/Llama-2-7B-Chat-GGUF","llama-2-7b-chat.Q2_K.gguf","./models")

    # Load the locally downloaded model here
    llm = CTransformers(
        model = mdpath,
        model_type="llama",
        temperature = 0.4,
        repetition_penalty = 1.2,
        top_k = 50,
        context_length = 6000
    )
    return llm

The function will use the previously defined global variable for prompt template and return a prompt object which will be used laters

In [16]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt


This function will create a "chain" object. You can consider chain object as collection point where LLM, vector store  and prompt merge together.

In [17]:
def retrieval_qa_chain(llm, prompt, db):     
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type='stuff',
                                           retriever=db.as_retriever(search_kwargs={'k': 2}),
                                           return_source_documents=True,
                                           chain_type_kwargs={'prompt': prompt}                                        
                                           )     
    return qa_chain

This function prepares all the required objects will be be needed for chain.

In [18]:
def qa_bot():
    # you can use a different embedding by replacing with model_name = sentence-transformers/all-MiniLM-L6-v2. system will download it during first execution
    embeddings = hugging_embedding()
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa


This line of code demonstrate loading of an llm object by using already defined functions. During the very first execution of this line, the specific LLM will be downloaded.

In [ ]:
llm = load_llm()

Ask question with out using any context. Here we are just asking a question directly to LLM and the response will be based on the knowledge which LLM is already having.

In [ ]:
llm("Q: Challenges of DevSecOps? A: ", max_tokens=32,  stop=["Q:", "\n"])

Calling the required function to create the vector database. During the very first execution of this code, the specific embedding model from huggingface will be downloaded and vector store will be created

In [ ]:
create_vector_db()

Load the chain to ask question

In [ ]:
chain = qa_bot()

Now we are asking a question by using the chain so that we can get a response based on the document which we provided as context.

In [61]:
query = "Challenges of DevSecOps"
chain(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'Challenges of DevSecOps',
 'result': 'Some challenges in incorporating security into a DevSecOps environment include:\n-Lack of awareness about the importance of security in DevOps\n-Insufficient resources and budget for security tools and training\n-Difficulty integrating security processes with existing workflows \n-Piecemeal approach to security, rather than holistic approach \n-Difficulty in measuring the effectiveness of security controls.',
 'source_documents': [Document(page_content='development in a DevOps \nenvironmentEnable Security \nAutomation\nAutomated security tools in the \nDevSecOps pipeline improves \noverall security by reducing \nvulnerabilities and security flaws \ndue to human error Automated Audit \nEvidence Collection  \nSecurity monitoring and notification \nsystems in DevSecOps creates an \nautomated audit trail throughout \nthe software development lifecycle,', metadata={'source': 'localdata/DevSecOps-Explained.pdf', 'page': 5}),
  Document(page_co

Now, lets play a little bit with documents and chunks. RAG provides the mechanims to ingest the knwoldge to an LLM. One of the most important step in the process is to create useful context for the LLM. LLLM have limited capacity and we can't feed full document  as context. We usually create several chunks of a document first. And then perform search based on our query to select the most appropriate contextual knowldge for the LLM.

In [ ]:
loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                     chunk_overlap=80)
texts = text_splitter.split_documents(documents)
len(texts)


Here we are searching our vector database with a smaple text. We are using similarity search to find the most similar chunks with our question. The out put of the will contain the chunks along with the score. the lower the score is, the better the search result is.

In [42]:
embeddings = hugging_embedding()

db = FAISS.from_documents(texts, embeddings)

docs_and_scores = db.similarity_search_with_score("challenges of  devsecops")
docs_and_scores


2023-12-26 18:42:14 - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[(Document(page_content='require development \nknowledge\nDevSecOps just means \ncode scanning\nDevSecOps prevents \norganizations from meeting their \nbusiness objectives', metadata={'source': 'localdata/DevSecOps-Explained.pdf', 'page': 3}),
  0.37491366),
 (Document(page_content='Copyright © 2018 Deloitte Development LLC. All rights reserved. 4Perceived challenges and piece- meal integration often hinder organizations from realizing the value of \nincorporating security into DevOpsCommon myths and misconceptions\nDevSecOps is incompatible \nwith my compliance \nrequirementsDevSecOps is only “Security as Code” \nor Automation\nDevSecOps requires \nsignificant tool investmentDevSecOps requires developers to be security expertsSecurity team does not \nrequire development \nknowledge', metadata={'source': 'localdata/DevSecOps-Explained.pdf', 'page': 3}),
  0.4234941),
 (Document(page_content='operations for a productEstablish Policies \nand Procedures\nIntroducing DevSecOps \nspecific p